In [7]:
import pandas as pd

Emission_data = pd.read_excel("Emissions_Data_Analysis.xlsx", sheet_name="Data", header=0)
print(Emission_data.head(3))

   CompanyID OperatingCountry             GICS_Sector  \
0  MFID_0001               FI  Consumer Discretionary   
1  MFID_0002               US             Industrials   
2  MFID_0003               US             Industrials   

                            GICS_Industry Group            GICS_Industry  \
0  Consumer Discretionary Distribution & Retail         Broadline Retail   
1                                Transportation  Air Freight & Logistics   
2                                 Capital Goods                Machinery   

                               GICS_SubIndustry  MarketCap_USD  \
0                              Broadline Retail   9.009218e+08   
1                       Air Freight & Logistics   7.805002e+09   
2  Industrial Machinery & Supplies & Components   4.250030e+09   

  MarketCap_Class    Sales_USD  Scope12_CarbonEmissions  ...  \
0       Small-cap   302.918892             12417.319222  ...   
1         Mid-cap  8502.990086            397530.941716  ...   
2        

In [ ]:
# Find all columns for Scope 3 categories (1 to 15)
scope3_category_cols = [col for col in Emission_data.columns if col.startswith("Scope3_CarbonEmissions_Cat_")]

# Sum them to get total Scope 3 emissions from all 15 categories
Emission_data["Scope3_sum_categories"] = Emission_data[scope3_category_cols].sum(axis=1)

print(Emission_data[["Scope3_Sum_Categories"]])

      Scope3_Sum_Categories
0              6.086799e+04
1              1.728498e+06
2              7.420561e+05
3              1.483363e+06
4              3.620477e+07
...                     ...
8391           3.685801e+05
8392           1.565456e+07
8393           1.792645e+08
8394           2.399803e+07
8395           3.007605e+06

[8396 rows x 1 columns]


In [16]:
Emission_data['Scope3_Discrepancy'] = Emission_data['Scope3_CarbonEmissions_Estimated'] - Emission_data['Scope3_Sum_Categories']
Emission_data['Scope3_Relative_Discrepancy'] = Emission_data['Scope3_Discrepancy'] / (Emission_data['Scope3_CarbonEmissions_Estimated'] + 1e-6)

# Filter normal-range companies
mask = (
    (Emission_data['Scope3_CarbonEmissions_Estimated'] > 0) &
    (Emission_data['Scope3_Sum_Categories'] > 0) &
    (Emission_data['Scope3_Relative_Discrepancy'].abs() <= 0.2)
)
Emission_data = Emission_data[mask]

# How many companies are left after filtering
print(len(Emission_data))

8175


In [ ]:
# Adjusted Scope 3: weighted average of scope3 emissions favoring model estimate over category sum
Emission_data['Adjusted_Est_Scope3'] = (
    0.8 * Emission_data['Scope3_CarbonEmissions_Estimated'] +
    0.2 * Emission_data['Scope3_Sum_Categories']
)

# Compare to company reported emissions, take max value of reported and adjusted
Emission_data['Scope3_Emissions'] = Emission_data[['Adjusted_Est_Scope3', 'Scope3_CarbonEmissions_Reported']].max(axis=1)

# Total emissions from Scope 1, 2, and 3
Emission_data['Total_Emissions'] = Emission_data['Scope3_Emissions'] + Emission_data['Scope12_CarbonEmissions']

# Group by sector and sum total emissions
Sector_sum_of_total_emissions = Emission_data.groupby('GICS_Sector')['Total_Emissions'].sum().sort_values(ascending=False)

print(Sector_sum_of_total_emissions.head())
print(f"The highest sector is {Sector_sum_of_total_emissions.idxmax()} with total emissions of {Sector_sum_of_total_emissions.max()} tons CO2 emission.")

GICS_Sector
Energy                    3.074004e+10
Industrials               2.253556e+10
Materials                 1.893768e+10
Consumer Discretionary    1.275764e+10
Financials                1.066661e+10
Name: Total_Emissions, dtype: float64
The highest sector is Energy with total emissions of 30740042050.7063 tons CO2e.
